## CS324 Introduction Assignment

### Overview

In this assignment, you will gain familiarity with prompting a wide range of foundation models. Specifically, you’ll work with large language models (LLMs) such as [BLOOM](https://huggingface.co/bigscience/bloom) and [BLOOMZ](https://huggingface.co/bigscience/bloomz) (BigScience). Language model prompting is the process of providing a model with an input text, and then having the model generate a response as output. For example, one could provide a language model with the prompt "The sky is" and the model might generate the response "blue". 

Prompts can be used to complete a breadth of tasks such as summarization (e.g., “Summarize the following paragraph: \<paragraph written out here\> Summary:”) or extraction (e.g., “Extract the phone number from the user bio: \<bio written out here\>.”)


The assignment is divided into three parts:


1. **Task selection**: In the first part of the assignment, you will select a task and evaluation dataset, which you will use in the remainder of the assignment.

2. **Prompt development**: In the second part of the assignment, you will explore how to develop effective prompts for LLMs. As part of this process, you may encounter both the joys and pains of prompting.  
  * On one hand, the right prompt and model can magically solve a seemingly complicated and unrelated task (w.r.t. the model's *training objective*.)  
  * On the other, prompting can be a brittle technique, and it may require time, effort, and creativity to develop a sufficient prompt.

3. **Model comparisons**: In the final part of the assignment, you will compare the performance (qualitatively and quantitatively) of several LLMs. Model comparisons will be along two dimensions:

  * Model Size
  * Training Techniques (e.g., vanilla autoregressive vs. instruction-tuned) 

### Submission Instructions

Complete the following sections. Download and submit this notebook (as an .ipynb file) to Canvas by 11:59PM on 1/24/23. 


---
### Part 0: Notebook Set-up

##### 0. Connect to a GPU supported runtime.

Connect to a hosted runtime and change the runtime type to "GPU".

<!-- Runtime -> Change Runtime Type -> Select "GPU" under Hardware Accelerator -->

##### 1. Install dependencies
First, make sure you have the necessary dependencies installed. You will need to install the transformers and torch libraries, as well as the huggingface-datasets library. You can do this by running the following commands:

In [ ]:
!pip install transformers torch datasets  accelerate bitsandbytes

##### 2. Import necessary libraries

Next, import the necessary libraries into your Jupyter notebook. You will need to import the following libraries

In [ ]:
import transformers
import torch
from datasets import load_dataset
from transformers import pipeline

---
### Part 1: Task, dataset, and evaluation metric selection [5 points]

In the first part of the assignment, you will select a task (e.g., sentiment classification) which you will use for the remainder of this assignment. Use the dataset and tasks found in the HuggingFace datasets library (https://huggingface.co/datasets). Select a labeled dataset (e.g., IMDB sentiment classification) and identify an evaluation metric (e.g., F1-score). 
 

In [ ]:
""" Example code to load dataset """
sample_dataset = load_dataset("imdb", split="test")

**Load your dataset below**

In [ ]:
dataset = """Enter your code here"""

In 2-3 lines, describe your chosen task, dataset and evaluation metric:

**Answer**: 

---
### Part 2: Prompt Development [30 points]

In the second part of the assignment, you will develop effective prompts for the foundation model. A prompt is a short sentence or phrase that serves as the starting point for the model's text generation (e.g., “Is the review positive or negative? \<review written here\>”). 

You will try to come up with a prompting "template" that allows a foundation model to correctly solve a task for 5 samples of your selected dataset.

For example, for sentiment classification of clothing reviews, we could specify a template as follows: 
```python
def template(sample_review):
    prompt = f"""
    Is this review positive or negative: 
    "Perfect dress and the customer service was awesome!"
    Sentiment: 
    positive
    ###
    Is this review positive or negative: 
    "not exact in sizing"
    Sentiment: 
    negative
    ###
    {sample_review}
    Sentiment:
    """
    return prompt
```
We can then apply the `template` function to 5 samples of our dataset, and feed the resulting prompt to a large language model. The hope is that the LLM processes the prompt as input, and outputs the desired `positive` or `negative` classification as natural text, via standard autoregressive language modeling.  


**To begin**, first select 5 samples from your chosen dataset.

In this section we will develop prompts for HuggingFace Bloom's 1.7B parameter model (a large language model with... 1.7 billion parameters). 

We'll first walk through some initial setup and two prompting strategies, before you come up with your own templates.

#### 1. Load the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = "bigscience/bloom-1b7"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)

#### 2. Configure a HuggingFace pipeline to generate text

Here, we will use the HuggingFace pipeline library to generate text using the model we have loaded

In [ ]:
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

#### 3. Generate Text 
Now we will now begin prompting the model. We will begin by walking you through a few examples of different prompt formats.

In [ ]:
# sample five entries from the sample dataset
samples = sample_dataset.to_pandas().sample(n=5)
sample_review = samples.iloc[0].text
print(sample_review)

#### 3.1 Prompt Technique: Zero-shot prompting

In this technique, an instruction for the task is usually specified in natural language. The model is expected to following the specification and output a correct response, **without any examples** (hence "zero shots").

In [ ]:
# Write a zero-shot prompt
prompt = f"""Classify the following movie review as positive or negative

Review: {sample_review}
Sentiment:"""

print(prompt)

In [ ]:
# Feed prompt to model to generate an output
output = generator(prompt, max_new_tokens=20)
print(output[0]['generated_text'])

#### 3.1 Prompt Technique: Few-shot prompting

In this technique, we provide a **few examples** in the prompt, optionally including task instructions as well (all as natural language). Even without said instructions, our hope is that the LLM can use the examples to autoregressively complete what comes next to solve the desired task. 


In [ ]:
""" 
Write a few-shot prompt. Here we include a few in-context examples to the model 
demonstrating how to complete the tasks
"""

prompt = f"""Review: The movie was horrible
Sentiment: Negative

Review: The movie was the best movie I have watched all year!!!
Sentiment: Positive

Review: {sample_review}
Sentiment:"""

print(prompt)

In [ ]:
# Feed prompt to model to generate an output
output = generator(prompt, max_new_tokens=1)
print(output[0]['generated_text'])

#### **Now it's your turn!** 

Create and evaluate 3 different prompting templates for your task and the 5 previously selected samples. 

For each sample, your code should print out the LLM's output in response to the sample's corresponding prompt. 

You should also write a function to automatically determine if the output correctly solved the task for each sample, and report the overall evaluation metric for all five samples.

Don't worry about creating templates that solve the task for all samples (e.g., 100% accuracy for classification). But your templates should be reasonable to some extent, i.e., developed with the aim to solve the task for at least 3 or 4 examples.

In [ ]:
"""
Instructions: Randomly subselect 5 examples from your dataset

Enter your code below:
"""

In [ ]:
"""
Prompt #1: Enter your code below

** Hint: For ease of use, wrap your prompts in functions (see sample code below) ***

----- SAMPLE CODE ------

def prompt_1(review):
  prompt = f'''Review: The movie was horrible
  Sentiment: Negative

  Review: The movie was the best movie I have watched all year!!!
  Sentiment: Positive

  Review: The film was a disaster
  Sentiment: Negative

  Review: {review}
  Sentiment:'''
  return prompt

for i, row in samples.iterrows():
  print(f"Sample {i}:")
  print(generator(prompt_1(row['text']), max_new_tokens=1)[0]['generated_text'])

-------------------------

"""

**Performance of Prompt #1:** 

In [ ]:
"""
Prompt #2: Enter your code below
"""

**Performance of Prompt #2:**

In [ ]:
"""
Prompt #3: Enter your code below
"""

**Performance of Prompt #3:**

---
### Part 3. Model comparisons [30 points]
In the final part of the assignment, pick one of the prompting templates you designed in Part 2, and randomly select 20 samples from your dataset chosen in Part 1. 

Using this template and evaluation set, we will now compare the performance of different large language models. 

We will focus on two dimensions of comparisons: 
* Model Size
* Model Training Objectives

#### 1. Model Size Comparisons
Here we will compare the performance of our prior Bloom 1.7b model (`bloom-1b7`), and a larger LLM, [bloom-3B](https://huggingface.co/bigscience/bloom-3b) (`bloom-3b`) with... you guessed it, *3 billion* parameters.

In [ ]:
"""
Instructions: Randomly subselect 20 examples from your dataset

Enter your code below:
"""

In [ ]:
"""
Instructions:  
1. Using the bigscience/bloom-1b7 model we loaded in Part 2, generate task outputs for each of the 20 examples in the dataset
2. Save all the outputs and ground-truth labels to a dataframe
3. Compute the task performance, using the the provided ground-truth labels in the dataset, and the 
   evaluation metric you identified in Step 1.

Enter your code below:
"""

In [ ]:
"""
Instructions:  
1. Load the bigscience/bloom-3b model and generate task outputs for each of the 20 examples in the dataset
2. Save all the outputs and ground-truth labels to a dataframe
3. Compute the task performance, using the the provided ground-truth labels in the dataset, and the 
   evaluation metric you identified in Step 1.


Enter your code below:
"""

Instructions:  
1. Compare the the quantitative performance of bloom-3B and bloom-1b7 parameter model.
2. Using a few of the outputs generated by models, describe a 1-2 qualitiative differences in the outputs 

**Answer (Provide your answer here):**

#### 2. Model Training Objectives
Here we will compare the performance of bloom-3b and [bloomz-3b](https://huggingface.co/bigscience/bloomz-3b).

Bloomz 3B is an *instruction-finetuned* model (a model explicitly trained to follow instructions), trained with a Bloom 3B backbone. 

* For more information on Bloom 3B , please see its [Model Card](https://huggingface.co/bigscience/bloomz-3b) (also linked above). 

* For more information on instruction fine-tuning, check out the corresponding [paper](https://arxiv.org/abs/2211.01786).

In [ ]:
"""
Instructions:  
1. Load the bigsciece/bloomz-3b model and generate task outputs for each of the 20 examples in the dataset
2. Save all the outputs and ground-truth labels to a dataframe
3. Compute the task performance, using the the provided ground truth labels in the dataset, and the 
  evaluation metric you identified in Step 1.


Enter your code below:
"""

Instructions:  
1. Compare the the quantitative performance of bloom-3b and bloomz-3b.
2. Using a few of the outputs generated by models, describe 1-2 qualitiative differences in the outputs 

**Answer (Provide your answer here):**

---
#### Bonus for fun: ChatGPT

As an additional comparison, try the above evaluation with [ChatGPT](https://openai.com/blog/chatgpt/), a *closed-source* model trained by OpenAI with instruction-finetuning. 

For now, ChatGPT provides a free way to interact with "Open"AI's closed models. You can access it at [https://chat.openai.com/](https://chat.openai.com/) (you may need to create an account first).

How do the outputs of ChatGPT compare against the open-source BLOOM and BLOOMZ models?

---
#### Bonus for credit: ChatGPT does your homework
As an additional bonus, can you think of a way to get ChatGPT to complete this assignment?

In code or text blocks below, paste your input to ChatGPT, and the resulting output. 

Provide a short response describing your thoughts on using ChatGPT to complete this assignment, and any necessary changes to complete this assignment.

Additional bonus for actually running ChatGPT-generated output and completing this assignment in the code blocks below.


In [ ]:
### Enter your input prompts for ChatGPT:



In [ ]:
### Copy and paste the resulting ChatGPT output:



In [ ]:
### Describe your experience with ChatGPT, and any changes you'd make 
### to successfully complete this assignment with ChatGPT's outputs, 
### in the text box below:

In [ ]:
### Make any necessary changes to the output to successfully 
### run the code and complete this assignment in the code blocks below



### And that's it! Congrats on finishing your first assignment!

Remember to submit this completed notebook to Canvas to recieve credit for this assignment